In [41]:
import subprocess
import pandas as pd
from multiprocess import Process
import time

from plotly.offline import iplot
from plotly.subplots import make_subplots

# Constants

In [42]:
RESULTS_DIR = 'results'
FLOW_DIR = 'flow'
CWND_DIR = 'cwnd'
CONGESTION_MECHANISMS_PREFIX = 'ns3::Tcp'

CONGESTION_MECHANISMS = [
    'LinuxReno',
    'Vegas',
    'Ledbat',
    'Dctcp',
    'Cubic',
    
    # 'Bbr',
    # 'Bic',
    # 'HighSpeed',
    # 'Htcp',
    # 'Hybla',
    # 'Illinois',
    # 'Lp',
    # 'Scalable',
    # 'Veno',
    # 'WestwoodPlus',
    # 'Yeah'
]

# Generate Flow Stats

In [43]:
subprocess.call('.././ns3 build'.split()) 

def run_simulation(mechanism: str):
    sim_cmd = ['.././ns3 run double-bottleneck-link.cc --']
    sim_cmd.append(f'--transportProtocol={CONGESTION_MECHANISMS_PREFIX}{mechanism}')
    sim_cmd.append(f'--flowStatFile={RESULTS_DIR}/{FLOW_DIR}/{mechanism}_flow_stats.csv')

    sim_cmd = ' '.join(sim_cmd)
    print(sim_cmd.strip())

    subprocess.call(sim_cmd.strip().split())

process_list: list[Process] = []
for mechanism in CONGESTION_MECHANISMS:
    process = Process(target=run_simulation, args=(mechanism, ))
    process.start()
    process_list.append(process)
    time.sleep(1)

for process in process_list:
    process.join()

[  0%] Building CXX object scratch/CMakeFiles/scratch_double-bottleneck-link.dir/double-bottleneck-link.cc.o
[  1%] Building CXX object scratch/CMakeFiles/scratch_single-bottleneck-link.dir/single-bottleneck-link.cc.o
[  1%] Linking CXX executable "/Users/maganthseetharaman/Desktop/ICS 234/ns-3-dev/build/scratch/ns3-dev-single-bottleneck-link-default"
[  1%] Linking CXX executable "/Users/maganthseetharaman/Desktop/ICS 234/ns-3-dev/build/scratch/ns3-dev-double-bottleneck-link-default"
Finished executing the following commands:
cd cmake-cache; /usr/local/bin/cmake --build . -j 7 ; cd ..
.././ns3 run double-bottleneck-link.cc -- --transportProtocol=ns3::TcpLinuxReno --flowStatFile=results/flow/LinuxReno_flow_stats.csv
.././ns3 run double-bottleneck-link.cc -- --transportProtocol=ns3::TcpVegas --flowStatFile=results/flow/Vegas_flow_stats.csv
.././ns3 run double-bottleneck-link.cc -- --transportProtocol=ns3::TcpLedbat --flowStatFile=results/flow/Ledbat_flow_stats.csv
.././ns3 run double-bo

In [44]:
data_collection = []

flow_data = pd.DataFrame()
for mechanism in CONGESTION_MECHANISMS:
    flow_file = f'../{RESULTS_DIR}/{FLOW_DIR}/{mechanism}_flow_stats.csv'

    data = pd.read_csv(flow_file)
    data.insert(0, 'Algorithm', mechanism)
    
    data_collection.append(data)

data = pd.concat(data_collection, ignore_index=True)

data

,Algorithm,Flow ID,Time,Throughput,Packets Dropped,Fairness,Window Size
0,LinuxReno,0,0.101,116.1400,0,0.513965,1
1,LinuxReno,1,0.101,107.9860,0,0.513965,1
2,LinuxReno,2,0.101,100.9020,0,0.513965,1
3,LinuxReno,3,0.101,94.6908,0,0.513965,1
4,LinuxReno,0,0.201,34.8362,0,0.530581,547
...,...,...,...,...,...,...,...
11975,Cubic,3,59.801,11.4632,64,0.544573,42
11976,Cubic,0,59.901,11.3617,64,0.544584,40
11977,Cubic,1,59.901,11.4821,64,0.544584,40
11978,Cubic,2,59.901,11.4807,65,0.544584,40


In [47]:
metric = 'Fairness'
flow_id = 1 #data['Flow ID'].unique()

fig = make_subplots(shared_xaxes=True, shared_yaxes=True)

for mechanism in CONGESTION_MECHANISMS:
    mechanism_data = data[(data['Algorithm'] == mechanism) & (data['Flow ID'] == flow_id)]
    fig.add_scatter(x=mechanism_data['Time'], y=mechanism_data[metric], name=mechanism)

fig.update_layout(showlegend=True, template = 'simple_white')
fig.update_xaxes(title_text='Time (Sec)')
fig.update_yaxes(title_text=metric)

iplot(fig)

In [46]:
# # DEPRECATED

# import matplotlib.pyplot as plt
# metric = 'Throughput'
# flow_id = data['Flow ID'].unique()

# ax = []
# fig = []
# for _ in flow_id:
#     _fig, _ax = plt.subplots(figsize=(25, 10))
#     ax.append(_ax)
#     fig.append(_fig)

# plot_index = 0
# for mechanism in CONGESTION_MECHANISMS:
#     for flow in flow_id:
#         # Filter the data for the current data ID
#         mechanism_data = data[(data['Algorithm'] == mechanism) & (data['Flow ID'] == flow)]
#         # Create the plot
#         # mechanism_data = mechanism_data[mechanism_data['Time'] > 250]
#         ax[plot_index].plot(mechanism_data['Time'], mechanism_data[metric], label=mechanism)
        
#         # Add labels and legend to the plot
#         # ax[plot_index].title.set_text(f'Flow ID: {flow}')
#         ax[plot_index].set_xlabel('Time')
#         ax[plot_index].set_ylabel(metric)
#         ax[plot_index].set_yscale('log')
#         ax[plot_index].set_xscale('log')

#         plot_index += 1

#     plot_index = 0
    
# for f in fig:
#     f.legend()

# # Show the plot
# plt.show()
# plt.close()